# Repository Discovery

It is difficult, without specialized knowledge or dedicated inspection, to determine how repos on Github are related to one another.

Which projects (repos) are trying to solve the same problem? Which are in the same ecosystem? Which pre-date which others?

We've begun to study this problem using the abstract model of networks. Our approach is summarized by the neuroscience Hebbian mneumonic approximation: "fire together, wire together."
Said with greater context, groups of projects that share features (contributors, in this first exploration) are more likely than others to do similar things.

We expect projects in the container orchestration space to, pairwise, share more contributors together than with projects outside of that context, on average.

## This notebook's contribution

In this notebook, we attempt to solve a challenging problem:

**Given the contributors in a known kernel of projects within the same domain, identify relevant, previously censored, repos that those contributors have also taken part in.**

Similarly,

**Given a list of newly discovered potential ecosystem member repositories, identify those that are the most preeminent, filtering out those that are natural 'noise,' irrelevant to the studied ecosystem.**

# Setup

Import Python packages, load database access credentials, define connection to DB.

In [50]:
#IMPORTS
import pandas as pd
import sqlalchemy as salc
import plotly.express as px
import pyarrow
import json
import uuid

In [51]:
#DATABASE ACCESS

with open("wasm_creds.json") as config_file:
    config = json.load(config_file)

database_connection_string = f"postgresql+psycopg2://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': f"-csearch_path={'augur_data'}"})

In [52]:
# SWITCHES

LOCAL_DATA = True

# Known contributors

At the outset, we identified a kernel of projects that we are highly confident exist within our ecosystem of interest.

From this kernel of projects, we identified all of the contributors and ranked them based on how they interacted with one another.

Those who interact with many others in a valuable way are more highly ranked than those who never interact whatsover, and soforth.

The data below is the artifact from that analysis, where each row is: {contributor_id, contributor_importance_ranking}

In [53]:
df_known_contribs = pd.read_csv('contrib_pagerank_scores.csv')
df_known_contribs = df_known_contribs.rename(columns={"contrib_id": "cntrb_id"})
df_known_contribs.head()

,cntrb_id,normalized_pagerank_score
0,010002df-6100-0000-0000-000000000000,20.000000
1,010000fd-e400-0000-0000-000000000000,19.052353
2,01002a5e-b400-0000-0000-000000000000,17.138503
3,0100034c-5400-0000-0000-000000000000,16.901099
4,01003291-1b00-0000-0000-000000000000,16.831682


# Contribution event stream

Github provides an API endpoint from which one can query every event that occurs. For instance, if a contributor creates an issue on some repo, that event will be logged in the stream.

We want to use this stream to identify the set of repositories that our Known Contributors are also working in that weren't in the initial ecosystem kernel.

The database that we use collects this event stream so we can query it from there rather than from Github. This is much faster than relying on the public Github API. In the Augur database, the table with this information is the 'contributor_repo' table.

In [54]:
event_stream_query = salc.sql.text(
    f"""
        SET SCHEMA 'augur_data';
        SELECT 
            c.cntrb_id,
            c.event_id,
            c.created_at,
            c.cntrb_repo_id as repo_id,
            c.repo_git,
            c.repo_name,
            c.gh_repo_id,
            c.cntrb_category as event_type
        FROM
        contributor_repo c
    """)


if not LOCAL_DATA:
    with engine.connect() as conn:
        df_event_stream = pd.read_sql_query(event_stream_query, conn)
        
    with open("df_event_stream.parquet", "wb+") as f:
        df_event_stream.cntrb_id = df_event_stream.cntrb_id.astype(str)
        df_event_stream.to_parquet(f)
else:
    with open("df_event_stream.parquet", "rb+") as f:
        df_event_stream = pd.read_parquet(f)

## Event stream data

Below we see a summary of the data we have:

- 3.5 million events,
- earliest is 8/19/2022, most recent is 9/27/2023

In [55]:
#WHAT IS THE NATURE OF THE DATA?
df_event_stream.describe()

,event_id,created_at,repo_id,gh_repo_id
count,3.725197e+06,3725197,3.725197e+06,3.725197e+06
mean,3.083393e+10,2023-08-01 19:24:19.586107904,1.938252e+07,3.429606e+08
min,2.888731e+10,2022-02-07 19:37:07,1.000000e+00,1.000000e+00
25%,3.030225e+10,2023-07-10 06:47:02,9.343510e+05,1.150397e+08
50%,3.095659e+10,2023-08-07 21:55:36,1.959492e+06,3.342359e+08
75%,3.139628e+10,2023-08-27 00:29:31,4.872971e+07,5.851590e+08
max,3.215458e+10,2023-09-27 20:13:13,5.366886e+07,6.974606e+08
std,8.201160e+08,NaN,2.233892e+07,2.361204e+08


### Head of dataframe

In [56]:
df_event_stream.head()

,cntrb_id,event_id,created_at,repo_id,repo_git,repo_name,gh_repo_id,event_type
0,01000ab1-4000-0000-0000-000000000000,31904121647,2023-09-18 09:49:57,48860218,https://api.github.com/repos/oxidecomputer/hubris,oxidecomputer/hubris,252314296,WatchEvent
1,01009668-df00-0000-0000-000000000000,31180979155,2023-08-17 10:35:15,34198358,https://api.github.com/repos/IAMSUPERMONKEY/fl...,IAMSUPERMONKEY/flutter_easyrefresh_v2_flutter3,500325907,WatchEvent
2,01004c0d-e400-0000-0000-000000000000,32146216302,2023-09-27 14:41:13,49521902,https://api.github.com/repos/WebAssembly/profiles,WebAssembly/profiles,413775155,IssueCommentEvent
3,01004c0d-e400-0000-0000-000000000000,32146168254,2023-09-27 14:39:44,49521903,https://api.github.com/repos/WebAssembly/relax...,WebAssembly/relaxed-simd,348477492,IssueCommentEvent
4,01009668-df00-0000-0000-000000000000,31180261079,2023-08-17 10:07:09,34198359,https://api.github.com/repos/Fitem/flutter_uti...,Fitem/flutter_util_code,647552683,WatchEvent


### Most common types of events

In [57]:
event_counts = df_event_stream.event_type.value_counts()
event_counts

event_type
PushEvent                        1336715
IssueCommentEvent                 518100
PullRequestEvent                  372087
PullRequestReviewEvent            326435
CreateEvent                       285811
PullRequestReviewCommentEvent     250293
WatchEvent                        218398
DeleteEvent                       172076
IssuesEvent                       161604
ForkEvent                          37986
ReleaseEvent                       19684
CommitCommentEvent                 12083
GollumEvent                         6629
MemberEvent                         4106
PublicEvent                         3190
Name: count, dtype: int64

In [58]:
fig_ec = px.bar(
    data_frame=event_counts.to_frame(),
    color=event_counts.to_frame().index,
)
fig_ec

### Repositories with the most events 

In [59]:
repo_event_counts = df_event_stream.repo_git.value_counts()
repo_event_counts

repo_git
https://api.github.com/repos/llvm/llvm-project                                                               42759
https://api.github.com/repos/pytorch/pytorch                                                                 12366
https://api.github.com/repos/rust-lang/rust                                                                  12228
https://api.github.com/repos/dotnet/runtime                                                                  11978
https://api.github.com/repos/intel/llvm                                                                      10008
                                                                                                             ...  
https://api.github.com/repos/graue/esofiles                                                                      1
https://api.github.com/repos/OvermindDL1/gt6-new-workspace                                                       1
https://api.github.com/repos/expln/metamath-lamp                       

In [60]:
df_fig_rec = repo_event_counts.to_frame() 
df_fig_rec = df_fig_rec[df_fig_rec["count"] > 2500]

fig_rec = px.bar(
    data_frame=df_fig_rec,
    color=df_fig_rec.index
)
fig_rec

## Contribution event stream re: Known contributors

We'd like to know which projects our group of contributors are the most active in. 

We'll cross-reference our event stream with the list of contributor IDs we have from our previous analysis.

In [61]:
UUID_known = df_known_contribs.cntrb_id.to_list()
UUID_known

['010002df-6100-0000-0000-000000000000',
 '010000fd-e400-0000-0000-000000000000',
 '01002a5e-b400-0000-0000-000000000000',
 '0100034c-5400-0000-0000-000000000000',
 '01003291-1b00-0000-0000-000000000000',
 '01019668-3400-0000-0000-000000000000',
 '01000e6f-b800-0000-0000-000000000000',
 '01000123-4b00-0000-0000-000000000000',
 '0100580b-d200-0000-0000-000000000000',
 '010003d7-c900-0000-0000-000000000000',
 '010044b7-6b00-0000-0000-000000000000',
 '01000844-b800-0000-0000-000000000000',
 '01000d33-f800-0000-0000-000000000000',
 '01027f51-2a00-0000-0000-000000000000',
 '010005fd-cc00-0000-0000-000000000000',
 '01005d27-9500-0000-0000-000000000000',
 '010001cf-5a00-0000-0000-000000000000',
 '010109e8-ab00-0000-0000-000000000000',
 '01000058-b400-0000-0000-000000000000',
 '01002912-dc00-0000-0000-000000000000',
 '010001ad-c700-0000-0000-000000000000',
 '010004fe-4500-0000-0000-000000000000',
 '0100319a-6800-0000-0000-000000000000',
 '010014b5-5700-0000-0000-000000000000',
 '0102f65a-0500-

In [62]:
# only consider events from known contributors.
df_known_event_stream = df_event_stream[df_event_stream.cntrb_id.isin(UUID_known)]

### Difference in size:

Whereas we initially considered 3.7 million events, for those contributors we've identified, we only consider 370k events, an order of magnitude reduction.

In [63]:
print(f"Diff in size: {df_event_stream.shape} compared to {df_known_event_stream.shape}")

Diff in size: (3725197, 8) compared to (370440, 8)


## Most common events among known contributors

In [64]:
kc_event_counts = df_known_event_stream.event_type.value_counts()
fig_kec = px.bar(
    data_frame=kc_event_counts.to_frame(),
    color=kc_event_counts.to_frame().index,
)
fig_kec

## Most common repositories among known contributors

In [66]:
kc_repo_event_counts = df_known_event_stream.repo_git.value_counts()
df_fig_rec_kc = kc_repo_event_counts.to_frame() 

fig_rec_kc = px.bar(
    data_frame=df_fig_rec_kc[:50],
    color=df_fig_rec_kc[:50].index
)
fig_rec_kc

## Interpretaion

We find here that some obvious WASM-specific repositories are contributed to most frequently by the contributors we already knew about.

However, as one might expect, repos that are popular among the general public like 'risingwavelabs/risingwave', (which is 5th in this list and 74th among the general population, despite being a distributed streaming DB (not about WASM))
are also popular among this slice of the general population.

Our challenge now is to find those repos that are *especially* popular among this population as compared to the popularity among the general public.

# Identify disproportionately popular repos

## Unknown contributors

We want to consider the popularity of repos among people who aren't in our initial set.

If our 'known' group of contributors is the subpopulation "p" and the general population is "P",
then the group of people who are in the general population but aren't in our sub-population is "P - p" or " P^ ", pronounced P-hat.

In [67]:
# Get events made by those in P^

df_general_event_stream = df_event_stream[~df_event_stream.cntrb_id.isin(UUID_known)]
df_general_event_stream.shape

(3354757, 8)

In [68]:
# Count the events-per-repo of P^

g_repo_event_counts = df_general_event_stream.repo_git.value_counts().to_frame()
g_repo_event_counts

,count
repo_git,
https://api.github.com/repos/llvm/llvm-project,42063
https://api.github.com/repos/pytorch/pytorch,12357
https://api.github.com/repos/dotnet/runtime,10659
https://api.github.com/repos/intel/llvm,10008
https://api.github.com/repos/apple/swift,9441
...,...
https://api.github.com/repos/colyseus/colyseus-sdk7,1
https://api.github.com/repos/decentraland-scenes/colyseus-sdk7,1
https://api.github.com/repos/devSquad-study/2023-CS-Study,1


In [69]:
# Join the counts of P^ with those of p

joined_counts = g_repo_event_counts.join(
    other=kc_repo_event_counts.to_frame(),
    on="repo_git",
    how="left",
    lsuffix="_general",
    rsuffix="_known",
)

joined_counts = joined_counts.dropna()
joined_counts

,count_general,count_known
repo_git,,
https://api.github.com/repos/llvm/llvm-project,42063,696.0
https://api.github.com/repos/pytorch/pytorch,12357,9.0
https://api.github.com/repos/dotnet/runtime,10659,1319.0
https://api.github.com/repos/apple/swift,9441,129.0
https://api.github.com/repos/rust-lang/rust,8158,4070.0
...,...,...
https://api.github.com/repos/erleans/pgo,1,1.0
https://api.github.com/repos/epi2me-labs/wf-metagenomics,1,1.0
https://api.github.com/repos/madebr/setup-sdl,1,3.0


## Simple approach

In this figure we visualize the count of contributions from those in 'p' and 'P^' stacked on top of one another. 

Most of the most-contributed-to projects of those in 'p' are also most-contributed-to by members of 'p' (they are not also largely contributed to by those in 'P^').

Some notable visual exceptions are 'NixOS/nixpkgs,' 'dotnet/runtime,' 'astral-sh/ruff,' 'paritytech/substrate,' and 'llvm/llvm-project.'

In [70]:
fig_joined_counts = px.bar(
    data_frame=joined_counts
    .sort_values(by="count_known", ascending=False) # sort the joined_counts df by the number of contributions made by those in p
    [:50], # only take the first 50
    y=["count_general", "count_known"]
)
fig_joined_counts

## Most sophisticated approach

Consider those contributors who are the most central in the kernel of projects we consider at the outset.

We could reasonably assume that if they're very central in the kernel of projects, then they're very interested in the ecosystem we're considering.